In [1]:
import streamlit as st

In [3]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from PIL import Image


In [ ]:
orders = pd.read_csv('sample-orders.csv',sep=',')

In [ ]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [ ]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [ ]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [ ]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [ ]:
rfmSegmentation = rfmTable

In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [ ]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [ ]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [ ]:
def app():
    st.title('RFM Analysis Report')
    
    st.title('Dataset')
    st.write(rfmTable)
        
    st.title('RFM Customer Quantile')
    if st.button('Generate RFM Plot.'):
        st.write(quantiles)
    
    st.title('RFM Customer Classification')
    st.write(rfmSegmentation)
    
    st.title('RFM Customer Type')
    st.write('Best Customers')
    st.write('Loyal Customers')
    st.write('Potential Loyalists')
    st.write('Big Spenders')
    st.write('At Risk Customers')
    st.write('Can’t Lose Them')
    st.write('Recent Customers')
    st.write('Lost Cheap Customers')
    
    st.title('RFM Customer Segmentation Analysis')
    if st.button('Generate RFM Plot.'):
        img =  Image.open("images/RFM.PNG")
        st.image(img)
    